## Synthetic Dataset Creation

In [7]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from numpy.linalg import cholesky

np.random.seed(42)
random.seed(42)

# -------------------------------
# 1. Basic Constants
# -------------------------------
NUM_COMPANIES = 1000
NUM_ANALYSTS = 700
YEARS = 10
ESTIMATE_TYPES = ["Stock Price", "EBITDA", "EPS", "Revenue", "EBIT"]
SECTORS = ["Technology", "Healthcare", "Finance", "Energy", "Consumer Goods", "Industrial", "Real Estate"]
MARKET_CAPS = ["Large Cap", "Mid Cap", "Small Cap"]
HORIZONS = ["Q0", "Q1", "Q2"]

BANKS = [
    "JP Morgan", "Goldman Sachs", "Morgan Stanley", "Citi", "Bank of America",
    "Deutsche Bank", "UBS", "Credit Suisse", "Barclays", "Wells Fargo"
]

# Ranges for initial values and vol for each estimate
ESTIMATE_PARAMS = {
    "Stock Price": {"init": (5, 200), "vol": (0.1, 0.3)},
    "EBITDA": {"init": (1e7, 2e9), "vol": (0.05, 0.2)},
    "EPS": {"init": (0.1, 20), "vol": (0.05, 0.25)},
    "Revenue": {"init": (1e8, 1e10), "vol": (0.05, 0.15)},
    "EBIT": {"init": (5e6, 2e9), "vol": (0.05, 0.2)}
}

# -------------------------------
# 2. Calendar Setup (Daily with Quarters)
# -------------------------------
start_date = pd.Timestamp("2013-01-01")
end_date = pd.Timestamp("2023-01-01")
trading_days = pd.date_range(start=start_date, end=end_date, freq='B')
n_days = len(trading_days)

# Identify quarter-end days (last trading day each quarter)
quarter_ends = trading_days.to_series().groupby(trading_days.to_period("Q")).last().values
quarter_ends = pd.to_datetime(quarter_ends)  # ensure Timestamps

# We'll store daily indexes for quick lookup
day_to_idx = {day: i for i, day in enumerate(trading_days)}

# -------------------------------
# 3. Market Regimes & Sentiment
# -------------------------------
REGIME_NAMES = ["Bull", "Bear"]
REGIME_TRANSITION = {
    "Bull": {"Bull": 0.90, "Bear": 0.10},
    "Bear": {"Bull": 0.20, "Bear": 0.80}
}

def next_regime(current_regime):
    probs = REGIME_TRANSITION[current_regime]
    return np.random.choice(list(probs.keys()), p=list(probs.values()))

def generate_sentiment_series(n_days):
    """
    Creates a random-walk 'sentiment' factor around 0, with slight mean reversion.
    Positive => higher drift, negative => lower drift.
    """
    sentiment = np.zeros(n_days)
    for i in range(1, n_days):
        sentiment[i] = sentiment[i-1] + np.random.normal(0, 0.01)
        # mean-revert slightly
        sentiment[i] -= 0.001 * sentiment[i]
    return sentiment

market_sentiment = generate_sentiment_series(n_days)

# We'll store a daily regime array
market_regime = []
current_regime = "Bull"
for i in range(n_days):
    market_regime.append(current_regime)
    # once a month, we consider a possible regime switch
    if i > 0 and trading_days[i].day == 1:
        current_regime = next_regime(current_regime)
market_regime = np.array(market_regime)

def regime_drift_offset(regime):
    # e.g. in a Bull regime, we add +0.05 to annual drift, Bear => -0.02
    if regime == "Bull":
        return 0.05
    else:
        return -0.02

def sentiment_drift_offset(sent):
    # e.g. scale sentiment by 0.1 => offset
    return 0.1 * sent

# We'll also define a simple sector rotation effect: each year we pick a "favored" sector
# that gets an extra +0.03 drift, and a "disfavored" sector that gets -0.02
year_to_favored = {}
year_to_disfavored = {}
for year in range(start_date.year, end_date.year + 1):
    favored = np.random.choice(SECTORS)
    disfavored = np.random.choice([s for s in SECTORS if s != favored])
    year_to_favored[year] = favored
    year_to_disfavored[year] = disfavored

def sector_rotation_offset(sector, date):
    y = date.year
    if sector == year_to_favored[y]:
        return 0.03
    elif sector == year_to_disfavored[y]:
        return -0.02
    else:
        return 0.0

# -------------------------------
# 4. Company Data & Growth Profiles
# -------------------------------
LIFE_CYCLES = ["Growth", "Mature", "Decline"]
def life_cycle_drift(lifecycle):
    # Example: Growth => +0.05, Mature => +0.02, Decline => -0.01
    if lifecycle == "Growth":
        return 0.05
    elif lifecycle == "Mature":
        return 0.02
    else:
        return -0.01

companies = [f"Company{i+1}" for i in range(NUM_COMPANIES)]
company_profiles = {}

for c in companies:
    sector = np.random.choice(SECTORS)
    mc = np.random.choice(MARKET_CAPS)
    lc = np.random.choice(LIFE_CYCLES, p=[0.4, 0.4, 0.2])  # 40% growth, 40% mature, 20% decline
    init_vals = {}
    vol_vals = {}
    for est in ESTIMATE_TYPES:
        init_vals[est] = np.random.uniform(*ESTIMATE_PARAMS[est]["init"])
        vol_vals[est] = np.random.uniform(*ESTIMATE_PARAMS[est]["vol"])
    company_profiles[c] = {
        "Sector": sector,
        "MarketCap": mc,
        "LifeCycle": lc,
        "Initial": init_vals,
        "Volatility": vol_vals
    }

# -------------------------------
# 5. Macroeconomic & Sector Shocks
# -------------------------------
shock_factor_market = np.ones(n_days)  # factor that modifies drift for entire market
shock_factor_sector = np.ones((n_days, len(SECTORS)))  # sector-specific factor

def generate_shocks():
    """
    Each month, small chance of a macro shock or sector shock that modifies
    the daily drift for a short period (5-10 days).
    """
    for i in range(1, n_days):
        if trading_days[i].day == 1:
            # macro shock
            if np.random.rand() < 0.05:
                shock_len = np.random.randint(5, 11)
                shock_mag = np.random.uniform(-0.05, 0.05)  # can be up or down
                for j in range(i, min(i + shock_len, n_days)):
                    shock_factor_market[j] += shock_mag
            # sector shock
            if np.random.rand() < 0.1:
                sector_hit = np.random.choice(SECTORS)
                idx_sec = SECTORS.index(sector_hit)
                shock_len = np.random.randint(5, 11)
                shock_mag = np.random.uniform(-0.08, 0.08)
                for j in range(i, min(i + shock_len, n_days)):
                    shock_factor_sector[j, idx_sec] += shock_mag

generate_shocks()

# -------------------------------
# 6. Company-Specific Events
# -------------------------------
events = {day: {} for day in trading_days}

def generate_company_events():
    """
    Random events: earnings_surprise, M&A, product_launch.
    Each quarter, there's a small chance for each event.
    """
    for c in companies:
        for q_day in quarter_ends:
            if np.random.rand() < 0.05:
                events[q_day].setdefault(c, []).append("earnings_surprise")
            if np.random.rand() < 0.01:
                events[q_day].setdefault(c, []).append("M&A")
            if np.random.rand() < 0.02:
                if company_profiles[c]["Sector"] in ["Healthcare", "Consumer Goods"]:
                    events[q_day].setdefault(c, []).append("product_launch")

generate_company_events()

def apply_company_event(c, day_idx, metric_values, metric, S):
    """
    Applies any event that occurs on this day for this company, possibly adjusting S.
    """
    d = trading_days[day_idx]
    if d in events and c in events[d]:
        for ev in events[d][c]:
            if ev == "earnings_surprise":
                jump_scale = 0.15 if metric == "Stock Price" else 0.10
                jump = np.random.uniform(-jump_scale, jump_scale)
                S *= (1 + jump)
            elif ev == "M&A":
                jump_scale = 0.2 if metric == "Stock Price" else 0.3
                jump = np.random.uniform(-jump_scale, jump_scale)
                S *= (1 + jump)
            elif ev == "product_launch":
                if metric in ["Revenue", "EBITDA", "EBIT"]:
                    jump = np.random.uniform(0.0, 0.1)
                    S *= (1 + jump)
                elif metric == "Stock Price":
                    jump = np.random.uniform(0.0, 0.05)
                    S *= (1 + jump)
                elif metric == "EPS":
                    jump = np.random.uniform(0.0, 0.08)
                    S *= (1 + jump)
    return S

# -------------------------------
# 7. Cross-Metric Correlation & Idiosyncratic Jumps
# -------------------------------
def generate_correlation_matrix(dim):
    A = np.random.normal(size=(dim, dim))
    cov = np.dot(A, A.T)  # random SPD matrix
    d = np.sqrt(np.diag(cov))
    corr = cov / np.outer(d, d)
    return corr

def jump_diffusion_poisson(lam=0.0002):
    """
    Very rare jump. If triggered, random magnitude in [-10%, +10%].
    """
    if np.random.rand() < lam:
        jump = np.random.uniform(-0.1, 0.1)
        return (1 + jump)
    return 1.0

company_metric_corr = {}
for c in companies:
    dim = len(ESTIMATE_TYPES)
    corr = generate_correlation_matrix(dim)
    try:
        L = cholesky(corr)
    except np.linalg.LinAlgError:
        L = np.eye(dim)
    company_metric_corr[c] = L

# -------------------------------
# 8. Seasonal Patterns for Some Metrics
# -------------------------------
def seasonal_factor(metric, sector, day):
    if metric == "Revenue" and sector == "Consumer Goods":
        if day.quarter == 4:
            return 1.05
    return 1.0

# -------------------------------
# 9. Analyst Coverage & Behavior
# -------------------------------
analysts = [f"Analyst{i+1}" for i in range(NUM_ANALYSTS)]

# Assign each analyst a random start date within the simulation period
analyst_start_date = {
    a: np.random.choice(trading_days[:int(n_days * 0.8)])  # 80% of range to ensure enough data
    for a in analysts
}

# We now define coverage so that Large Caps have more coverage, Small Caps fewer
coverage = {}
for c in companies:
    coverage[c] = []
    mc = company_profiles[c]["MarketCap"]
    if mc == "Large Cap":
        min_cov, max_cov = 15, 25
    elif mc == "Mid Cap":
        min_cov, max_cov = 8, 15
    else:  # Small Cap
        min_cov, max_cov = 3, 8
    possible_analysts = analysts
    num_cov = np.random.randint(min_cov, max_cov + 1)

    if len(possible_analysts) >= num_cov:
        covered_analysts = np.random.choice(possible_analysts, size=num_cov, replace=False)
    elif len(possible_analysts) > 0:
        # fallback: sample with replacement or just use all available analysts
        covered_analysts = np.random.choice(possible_analysts, size=min(len(possible_analysts), num_cov), replace=True)
    else:
        # if literally no analysts are available, skip coverage for this company
        continue

    coverage[c].extend(covered_analysts)


def update_coverage_each_year():
    """
    Simulates analysts dropping and picking up coverage over time.
    """
    for c in companies:
        current_analysts = coverage[c]
        new_analysts = []

        # Drop some analysts randomly (15% churn)
        for a in current_analysts:
            if np.random.rand() > 0.85:  # 15% chance to drop coverage
                continue
            new_analysts.append(a)

        # Possibly add new analysts (1-3 per company)
        needed = np.random.randint(1, 4)
        while len(new_analysts) < len(current_analysts):
            # Select analysts not already covering this company
            candidates = [a for a in analysts if a not in new_analysts]
            if len(candidates) == 0:
                break  # no available new analysts left
            a_new = np.random.choice(candidates)
            new_analysts.append(a_new)
            if len(new_analysts) >= len(current_analysts) + needed:
                break

        coverage[c] = new_analysts

# ——————————————
# 9.a. Cohort-based Analyst Reliability
# ——————————————
# (remove any previous `stars = …` or Beta-draw loops)

# 1) define cohort sizes
num = len(analysts)
n_stars = int(0.30 * num)   # top 30%
n_duds  = int(0.15 * num)   # bottom 15%

# 2) randomly assign analysts into 3 cohorts
pool       = analysts.copy()
top_cohort = random.sample(pool, n_stars)
for a in top_cohort: pool.remove(a)
bot_cohort = random.sample(pool, n_duds)
for a in bot_cohort: pool.remove(a)
avg_cohort = pool  # the remaining 55%

# 3) set Beta parameters per cohort
cohort_params = {
    "top": (5, 1),   # clustered near 1.0
    "avg": (2, 2),   # around 0.5
    "bot": (1, 5),   # clustered near 0.0
}

analyst_reliability = {}
for a in analysts:
    if   a in top_cohort:  α, β = cohort_params["top"]
    elif a in bot_cohort:  α, β = cohort_params["bot"]
    else:                   α, β = cohort_params["avg"]

    draw = np.random.beta(α, β)
    # scale into [0.3, 0.95] so even duds aren’t always zero
    base_rel = 0.3 + 0.65 * draw

    # assign the same base_rel for every (sector, market-cap, metric)
    analyst_reliability[a] = {
        (s, mc, est): base_rel
        for s  in SECTORS
        for mc in MARKET_CAPS
        for est in ESTIMATE_TYPES
    }

analyst_learning_rate = {a: np.random.uniform(-0.01, 0.02) for a in analysts}
# Initial experience level (0 to 1), starts low
analyst_experience = {a: np.random.uniform(0.1, 0.4) for a in analysts}

# Analyst forecast bias: some are consistently optimistic (overestimate), some pessimistic
# Analyst forecast bias: wider spread of optimism/pessimism (±5% σ)
analyst_bias = {}
for a in analysts:
    analyst_bias[a] = np.random.normal(loc=0.0, scale=0.05)

# Some analysts are stubborn and won't update their bias
stubborn_analysts = set(np.random.choice(analysts, size=int(0.1 * NUM_ANALYSTS), replace=False))

def herding_adjustment(predictions, weight=0.05):
    """Pull each forecast only 5% toward the group mean, preserving dispersion."""
    if not predictions:
        return []
    mean_pred = np.mean(predictions)
    return [p + weight * (mean_pred - p) for p in predictions]

def update_reliability_on_surprise(analyst, sector, mc, est, error):
    base_rel = analyst_reliability[analyst][(sector, mc, est)]
    if abs(error) > 0.3:
        base_rel -= 0.02
    base_rel = max(0.3, min(0.99, base_rel))
    analyst_reliability[analyst][(sector, mc, est)] = base_rel

# -------------------------------
# 10. Putting it All Together (Daily Simulation + Quarterly Predictions)
# -------------------------------
real_paths = {c: {} for c in companies}
for c in companies:
    for est in ESTIMATE_TYPES:
        real_paths[c][est] = np.zeros(n_days)

# Initialize from "Initial"
for c in companies:
    for est in ESTIMATE_TYPES:
        real_paths[c][est][0] = company_profiles[c]["Initial"][est]

# Daily simulation
for i in range(1, n_days):
    date = trading_days[i]
    dt = 1/252.0
    reg = market_regime[i]
    sentiment_val = market_sentiment[i]

    # We'll apply macro/sector shocks too
    macro_factor = shock_factor_market[i]

    for c in companies:
        sector = company_profiles[c]["Sector"]
        mc = company_profiles[c]["MarketCap"]
        lc = company_profiles[c]["LifeCycle"]

        L = company_metric_corr[c]
        dim = len(ESTIMATE_TYPES)

        z = np.random.normal(0, 1, dim)
        corr_z = L.dot(z)

        lc_drift = life_cycle_drift(lc)
        regime_offset = regime_drift_offset(reg)
        sent_offset = sentiment_drift_offset(sentiment_val)
        sector_rot = sector_rotation_offset(sector, date)

        # Combine into an annual drift, then scale by shock_factor
        annual_drift = (lc_drift + regime_offset + sent_offset + sector_rot)
        # Apply macro factor and sector factor
        idx_sec = SECTORS.index(sector)
        # e.g. multiply drift by shock factors
        annual_drift *= macro_factor
        annual_drift *= shock_factor_sector[i, idx_sec]

        for idx_est, est in enumerate(ESTIMATE_TYPES):
            S_prev = real_paths[c][est][i-1]
            vol = company_profiles[c]["Volatility"][est]

            daily_drift = annual_drift / 252.0
            dW = corr_z[idx_est] * np.sqrt(dt)

            # Jump diffusion
            jump_factor = jump_diffusion_poisson(lam=0.0002)

            daily_ret = daily_drift - 0.5*(vol**2)*dt + vol*dW
            daily_ret = np.clip(daily_ret, -0.03, 0.03)

            S_new = S_prev * np.exp(daily_ret) * jump_factor
            S_new *= seasonal_factor(est, sector, date)
            S_new = apply_company_event(c, i, company_profiles[c], est, S_new)

            real_paths[c][est][i] = S_new

# Build final DataFrame of predictions with a "reporting lag"
records = []
revealed_values = {c: {est: {} for est in ESTIMATE_TYPES} for c in companies}

for q_idx, q_day in enumerate(quarter_ends):
    q_label = f"Q{q_day.quarter} {q_day.year}"

    if q_idx < len(quarter_ends) - 1:
        reveal_day = quarter_ends[q_idx+1]
    else:
        reveal_day = None

    if q_day not in day_to_idx:
        continue
    q_idx_day = day_to_idx[q_day]

    # Real values at quarter-end
    for c in companies:
      for est in ESTIMATE_TYPES:
        for horizon in HORIZONS:
            horizon_offset = {
                "Q0": 0,
                "Q1": 1,
                "Q2": 2,
            }.get(horizon, 0)

            if q_idx + horizon_offset >= len(quarter_ends):
                continue  # skip if horizon goes past simulation range

            target_day = quarter_ends[q_idx + horizon_offset]
            target_idx = day_to_idx.get(target_day, None)
            if target_idx is None:
                continue

            real_val = real_paths[c][est][target_idx]
            revealed_values[c][est][horizon] = real_val

    from pandas.tseries.offsets import BMonthEnd
    revision_days = [
      q_day - BMonthEnd(3),
      q_day - BMonthEnd(2),
      q_day - BMonthEnd(1)
    ]

    # Add initial forecasts BEFORE any revision
    rev_label = "Initial"

    for c in companies:
        sector = company_profiles[c]["Sector"]
        mc = company_profiles[c]["MarketCap"]
        coverage_list = coverage[c]

        for est in ESTIMATE_TYPES:
            for horizon in ["Q0", "Q1", "Q2"]:
                horizon_offset = {
                    "Q0": 0,
                    "Q1": 1,
                    "Q2": 2
                }[horizon]

                if q_idx + horizon_offset >= len(quarter_ends):
                    continue
                target_day = quarter_ends[q_idx + horizon_offset]
                target_idx = day_to_idx.get(target_day, None)
                if target_idx is None:
                    continue
                real_val = real_paths[c][est][target_idx]

                init_preds = []
                active_analysts = []
                for a in coverage_list:
                    if q_day >= analyst_start_date[a] and np.random.rand() < 0.8:
                        active_analysts.append(a)

                for a in active_analysts:
                    base_rel = analyst_reliability[a][(sector, mc, est)]
                    # wider base noise plus per-analyst rookies boost
                    noise_vol = np.random.uniform(0.1, 0.4)
                    error_std = (1 - base_rel) * noise_vol
                    error_factor = np.random.normal(1 + analyst_bias[a], error_std)
                    init_pred = real_val * error_factor
                    init_preds.append(init_pred)

                final_preds = herding_adjustment(init_preds)

                for idx_a, a in enumerate(active_analysts):
                    pred_val = final_preds[idx_a]
                    records.append([
                        q_label, est, pred_val, None,
                        c, a, np.random.choice(BANKS),
                        company_profiles[c]["MarketCap"],
                        sector,
                        analyst_start_date[a],
                        horizon,
                        rev_label  # this will say "Initial"
                    ])

    for rev_day in revision_days:
        if rev_day not in day_to_idx:
            continue
        rev_label = f"Rev {rev_day.strftime('%Y-%m-%d')}"

        for c in companies:
            sector = company_profiles[c]["Sector"]
            mc = company_profiles[c]["MarketCap"]
            coverage_list = coverage[c]

            for est in ESTIMATE_TYPES:
                for horizon in ["Q0", "Q1", "Q2"]:  # only short-term forecasts

                    horizon_offset = {
                        "Q0": 0,
                        "Q1": 1,
                        "Q2": 2
                    }[horizon]

                    if q_idx + horizon_offset >= len(quarter_ends):
                        continue
                    target_day = quarter_ends[q_idx + horizon_offset]
                    target_idx = day_to_idx.get(target_day, None)
                    if target_idx is None:
                        continue
                    real_val = real_paths[c][est][target_idx]

                    init_preds = []
                    active_analysts = []
                    for a in coverage_list:
                        if rev_day >= analyst_start_date[a] and np.random.rand() < 0.8:
                            active_analysts.append(a)

                    for a in active_analysts:
                        base_rel = analyst_reliability[a][(sector, mc, est)]
                        noise_vol = np.random.uniform(0.1, 0.4)
                        error_std = (1 - base_rel) * noise_vol
                        error_factor = np.random.normal(1 + analyst_bias[a], error_std)
                        init_pred = real_val * error_factor
                        init_preds.append(init_pred)

                    final_preds = herding_adjustment(init_preds)

                    for idx_a, a in enumerate(active_analysts):
                        pred_val = final_preds[idx_a]
                        records.append([
                            q_label, est, pred_val, None,
                            c, a, np.random.choice(BANKS),
                            company_profiles[c]["MarketCap"],
                            sector,
                            analyst_start_date[a],
                            horizon,
                            rev_label,
                        ])

    # Possibly update coverage each year
    if q_day.quarter == 1:
        update_coverage_each_year()

df = pd.DataFrame(records, columns=[
    "Quarter", "Estimate Type", "Predicted Value", "Real Value",
    "Company", "Analyst", "Bank", "Market Cap", "Sector", "Start Date", "Horizon", "Revision Time"
])

# Fill in Real Value for each quarter
real_value_by_quarter = {}
for q_day in quarter_ends:
    q_label = f"Q{q_day.quarter} {q_day.year}"
    if q_day in day_to_idx:
        idx_day = day_to_idx[q_day]
        for c in companies:
            for est in ESTIMATE_TYPES:
                real_value_by_quarter[(q_label, c, est)] = real_paths[c][est][idx_day]

for i in range(len(df)):
    row = df.iloc[i]
    q_label = row["Quarter"]
    c = row["Company"]
    est = row["Estimate Type"]
    horizon = row["Horizon"]

    if c in revealed_values and est in revealed_values[c] and horizon in revealed_values[c][est]:
        df.at[i, "Real Value"] = revealed_values[c][est][horizon]

# Final pass: update reliability based on errors
for i in range(len(df)):
    row = df.iloc[i]
    c = row["Company"]
    est = row["Estimate Type"]
    sector = company_profiles[c]["Sector"]
    mc = company_profiles[c]["MarketCap"]
    a = row["Analyst"]
    pred = row["Predicted Value"]
    rv = row["Real Value"]
    if rv is not None and rv != 0:
        error_ratio = (pred - rv) / rv
        update_reliability_on_surprise(a, sector, mc, est, error_ratio)

        # also apply learning based on experience
        base_rel = analyst_reliability[a][(sector, mc, est)]
        experience = analyst_experience[a]
        learning_rate = analyst_learning_rate[a]

        # Experience modulates the effect of learning
        delta = learning_rate * experience
        new_rel = base_rel + delta
        new_rel = max(0.3, min(0.99, new_rel))
        analyst_reliability[a][(sector, mc, est)] = new_rel

        # Analyst gains experience gradually
        analyst_experience[a] = min(1.0, experience + 0.002)

        if a not in stubborn_analysts:
          # Adjust bias based on error direction (analysts learn over time)
          bias_adj = -0.1 * error_ratio  # e.g., if error_ratio > 0, reduce positive bias
          analyst_bias[a] += bias_adj
          analyst_bias[a] = max(-0.1, min(0.1, analyst_bias[a]))  # clamp between [-10%, +10%]

df.head()

,Quarter,Estimate Type,Predicted Value,Real Value,Company,Analyst,Bank,Market Cap,Sector,Start Date,Horizon,Revision Time
0,Q1 2013,Stock Price,3.083182e+01,75.999668,Company5,Analyst513,Credit Suisse,Mid Cap,Technology,2013-02-19,Q0,Initial
1,Q1 2013,Stock Price,3.657365e+01,75.999668,Company5,Analyst513,Morgan Stanley,Mid Cap,Technology,2013-02-19,Q1,Initial
2,Q1 2013,Stock Price,4.628826e+01,75.999668,Company5,Analyst513,Barclays,Mid Cap,Technology,2013-02-19,Q2,Initial
3,Q1 2013,EBITDA,1.291998e+09,5748714647.472132,Company5,Analyst513,Barclays,Mid Cap,Technology,2013-02-19,Q0,Initial
4,Q1 2013,EBITDA,1.221465e+09,5748714647.472132,Company5,Analyst513,JP Morgan,Mid Cap,Technology,2013-02-19,Q1,Initial


In [8]:
# overwrite the Real Value column using the correct mapping
df["Real Value"] = df.apply(
    lambda row: real_value_by_quarter[
        (row["Quarter"], row["Company"], row["Estimate Type"])
    ],
    axis=1
)

In [9]:
df.head()

,Quarter,Estimate Type,Predicted Value,Real Value,Company,Analyst,Bank,Market Cap,Sector,Start Date,Horizon,Revision Time
0,Q1 2013,Stock Price,3.083182e+01,2.906252e+01,Company5,Analyst513,Credit Suisse,Mid Cap,Technology,2013-02-19,Q0,Initial
1,Q1 2013,Stock Price,3.657365e+01,2.906252e+01,Company5,Analyst513,Morgan Stanley,Mid Cap,Technology,2013-02-19,Q1,Initial
2,Q1 2013,Stock Price,4.628826e+01,2.906252e+01,Company5,Analyst513,Barclays,Mid Cap,Technology,2013-02-19,Q2,Initial
3,Q1 2013,EBITDA,1.291998e+09,1.264608e+09,Company5,Analyst513,Barclays,Mid Cap,Technology,2013-02-19,Q0,Initial
4,Q1 2013,EBITDA,1.221465e+09,1.264608e+09,Company5,Analyst513,JP Morgan,Mid Cap,Technology,2013-02-19,Q1,Initial


In [10]:
df["Revision Time"].unique()

array(['Initial', 'Rev 2013-01-31', 'Rev 2013-02-28', 'Rev 2013-03-29',
       'Rev 2013-04-30', 'Rev 2013-05-31', 'Rev 2013-06-28',
       'Rev 2013-07-31', 'Rev 2013-08-30', 'Rev 2013-09-30',
       'Rev 2013-10-31', 'Rev 2013-11-29', 'Rev 2013-12-31',
       'Rev 2014-01-31', 'Rev 2014-02-28', 'Rev 2014-03-31',
       'Rev 2014-04-30', 'Rev 2014-05-30', 'Rev 2014-06-30',
       'Rev 2014-07-31', 'Rev 2014-08-29', 'Rev 2014-09-30',
       'Rev 2014-10-31', 'Rev 2014-11-28', 'Rev 2014-12-31',
       'Rev 2015-01-30', 'Rev 2015-02-27', 'Rev 2015-03-31',
       'Rev 2015-04-30', 'Rev 2015-05-29', 'Rev 2015-06-30',
       'Rev 2015-07-31', 'Rev 2015-08-31', 'Rev 2015-09-30',
       'Rev 2015-10-30', 'Rev 2015-11-30', 'Rev 2015-12-31',
       'Rev 2016-01-29', 'Rev 2016-02-29', 'Rev 2016-03-31',
       'Rev 2016-04-29', 'Rev 2016-05-31', 'Rev 2016-06-30',
       'Rev 2016-07-29', 'Rev 2016-08-31', 'Rev 2016-09-30',
       'Rev 2016-10-31', 'Rev 2016-11-30', 'Rev 2016-12-30',
       'Rev 2

In [11]:
df.shape

(13744263, 12)